In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import os

import seaborn as sns

In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
filepath_isotypes = '/networks/cavd/VDCs/Schief/Schief_856-G002/SkinReactions/data/Glycan_array_Scripps/processed_data/DRAFT_CAVD_G002_Glycan_Microarray_data_processed_2024-10-16.txt'
df_glycan_isotypes = pd.read_csv(filepath_isotypes, sep="\t")

usecols = ['sample_id',
           'isotype', 
           'ptid', 
           'study_week',
           'spot_name', 
           'glycan_m_number', 
           'background_subtraced_mean_signal']

df = df_glycan_isotypes[usecols]
df.sample_id = df.sample_id.astype(str)

def centered_mean(x):
    if len(x) >= 6:
        return np.mean(np.sort(x)[1:-1])
    else:
        return np.mean(x)

df['centered_mean'] = df.groupby(['isotype','sample_id','glycan_m_number'])[['background_subtraced_mean_signal']].transform(centered_mean)
df = df.drop(columns='background_subtraced_mean_signal').drop_duplicates()

df = df.reset_index(drop=True)

calc_responses = df.copy()

calc_responses['threshold'] = 100
calc_responses['response_flag'] = calc_responses.centered_mean > 100
calc_responses["count_of_responses"] = calc_responses.groupby(['isotype','study_week','glycan_m_number'])[['response_flag']].transform('sum')
calc_responses["prop_of_responses"] = calc_responses.groupby(['isotype','study_week','glycan_m_number'])[['response_flag']].transform(lambda x: x.sum()/len(x))

calc_responses["overall_response_rate"] = calc_responses.groupby(['isotype','glycan_m_number'])[['response_flag']].transform(lambda x: x.sum()/len(x))
calc_responses = calc_responses.sort_values(by='overall_response_rate', ascending=False)

In [ ]:
import scipy.stats as stats

data = pd.pivot_table(calc_responses, 
               index=['isotype','ptid','glycan_m_number'],
               columns='study_week',
               values='centered_mean'
              ).reset_index()
data.columns = [i.lower().replace(" ","") for i in data.columns]

def concord(x,y):
    rho = stats.pearsonr(x,y).statistic
    sigma_x = np.var(x)
    sigma_y = np.var(y)
    mu_x = np.mean(x)
    mu_y = np.mean(y)

    return 2*rho*sigma_x*sigma_y/(sigma_x**2 + sigma_y**2 + (mu_x - mu_y)**2)


def get_concord(isotype, glycan, wk):
    # subset to reponders
    g = data.loc[(data.wk0 > 100) | (data[wk] > 100)]
    g = g.loc[(g.isotype==isotype) & (g.glycan_m_number==glycan) & (g.wk0.notna()) & (g[wk].notna())]
    x = g.wk0
    y = g[wk]
    if len(x) > 1 and len(y) > 1:
        return concord(x,y)
    else:
        return np.nan

concordance_table = calc_responses[['isotype','glycan_m_number','study_week','count_of_responses']].drop_duplicates()
concordance_table = pd.pivot(concordance_table, index=['isotype','glycan_m_number'], columns='study_week', values='count_of_responses')[['Wk 0', 'Wk 8', 'Wk 10']].reset_index()
concordance_table = concordance_table.rename(columns={i:f"{i} response_count".replace(" ","_").lower() for i in ['Wk 0', 'Wk 8', 'Wk 10']})

concordance_table["Wk 0/Wk 8"] = concordance_table.apply(lambda x: get_concord(x.isotype, x.glycan_m_number, 'wk8'), axis=1)
concordance_table["Wk 0/Wk 10"] = concordance_table.apply(lambda x: get_concord(x.isotype, x.glycan_m_number, 'wk10'), axis=1)

# concordance coeff. calculated for each isotype/glycan for each ppt that was a responder for at least one timepoint per comparison
concordance_table = concordance_table.loc[concordance_table['Wk 0/Wk 8'].notna() | concordance_table['Wk 0/Wk 10'].notna()]

concordance_table["for_sorting"] = concordance_table[['Wk 0/Wk 8','Wk 0/Wk 10']].max(axis=1)

concordance_table = concordance_table.sort_values(by=['isotype','for_sorting'], ascending=False).drop(columns='for_sorting')

In [11]:
savedir = '/networks/vtn/lab/SDMC_labscience/operations/documents/templates/assay/template_testing/glycan_analysis_11_26_2024/'
concordance_table.to_csv(savedir + "concordance_coeff.csv", index=False)

## kappa

In [14]:
agreement_inputs = calc_responses[['isotype','glycan_m_number','study_week','ptid','response_flag']].drop_duplicates()

In [15]:
agreement_inputs = pd.pivot(agreement_inputs, index=['isotype','glycan_m_number','ptid'], columns='study_week', values='response_flag')[['Wk 0','Wk 8','Wk 10']].reset_index()

In [18]:
def get_kappa(isotype, glycan):
    d = agreement_inputs.loc[(agreement_inputs.glycan_m_number==glycan) & (agreement_inputs.isotype==isotype)]
    d = d.loc[d['Wk 0'].notna()]
    
    v8 = cohen_kappa_score(d.loc[d['Wk 8'].notna(),'Wk 0'].astype(int), d.loc[d['Wk 8'].notna(),'Wk 8'].astype(int))
    v10 = cohen_kappa_score(d.loc[d['Wk 10'].notna(),'Wk 0'].astype(int), d.loc[d['Wk 10'].notna(),'Wk 10'].astype(int))
    return v8, v10

In [25]:
agreement = calc_responses[['isotype','glycan_m_number','study_week','count_of_responses']].drop_duplicates()
agreement = pd.pivot(agreement, index=['isotype','glycan_m_number'], columns='study_week', values='count_of_responses')[['Wk 0', 'Wk 8', 'Wk 10']].reset_index()
agreement = agreement.rename(columns={i:f"{i} response_count".replace(" ","_").lower() for i in ['Wk 0', 'Wk 8', 'Wk 10']})
agreement[["Wk0/Wk8","Wk0/Wk10"]] = agreement.apply(lambda x: get_kappa(x.isotype, x.glycan_m_number), axis=1).apply(pd.Series)

/home/bhaddock/miniconda3/envs/py10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/bhaddock/miniconda3/envs/py10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/bhaddock/miniconda3/envs/py10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/home/bhaddock/miniconda3/envs/py10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:730: RuntimeWarning: invalid value encountered in scalar divide
  k = np.sum(w_mat * confusion)